In [2]:
import pandas as pd
import numpy as np

In [6]:
projects = pd.read_csv("project_data.csv")
projects

,project_name,community,country,pv_size_kwh,customer_count,capex_usd
0,Kalobeyei Settlement,kalobeyei_settlement,Kenya,541.40,2755,967872
1,Ndeda,ndeda,Kenya,23.40,405,197853
2,Emereoke,emereoke,Nigeria,28.08,401,336197
3,Akipelai,akipelai,Nigeria,67.32,355,323701
4,Locheremoit,locheremoit,Kenya,37.00,329,311220
5,Oloibiri,oloibiri,Nigeria,67.32,321,346464
6,Ringiti,ringiti,Kenya,20.45,291,142561
7,Ozuzu,ozuzu,Nigeria,18.72,265,183502
8,Bendeghe-Afi,bendeghe_afi,Nigeria,18.72,246,218907
9,Opu,opu,Nigeria,14.04,227,198777


In [20]:
df = pd.read_csv("customers_info.csv")
df["tariff"].unique()

array(['Commercial', 'Residential', 'Religious Institution', 'Free', nan],
      dtype=object)

In [21]:
rep_map = {'kalobeyei town': 'Kalobeyei Town',
           'oyamo': 'Oyamo', 'ringiti': 'Ringiti', 'RIngiti': 'Ringiti',
           'Kalobeyei Settlement Village 3 - Yelele': 'Kalobeyei Settlement'}

df["projectName"].replace(rep_map, inplace=True)
df["projectName"].unique()

array(['Kalobeyei Settlement', nan, 'Kalobeyei Town', 'Ngurunit', 'Oyamo',
       'Olkiramatian', 'Kapelbok', 'Katiko', 'Lomekwi', 'Locheremoit',
       'Lorengelup', 'Nakukulas', 'Kangitan Kori', 'Kakuma 3 - Okapi',
       'Oloibiri', 'Akipelai', 'Ozuzu', 'Emereoke', 'Bendeghe-Afi',
       'Ekong Anaku', 'Opu', 'Balep', 'Test ThunderCloud', 'Ndeda',
       'Ringiti'], dtype=object)

In [35]:
df.groupby(by=["projectName"])["customerAccountNumber"].count()

projectName
Akipelai                 370
Balep                    196
Bendeghe-Afi             290
Ekong Anaku              199
Emereoke                 407
Kakuma 3 - Okapi         200
Kalobeyei Settlement    2867
Kalobeyei Town           143
Kangitan Kori            118
Kapelbok                 199
Katiko                   120
Locheremoit              332
Lomekwi                   74
Lorengelup                64
Nakukulas                217
Ndeda                    408
Ngurunit                 253
Olkiramatian              76
Oloibiri                 343
Opu                      233
Oyamo                    146
Ozuzu                    274
Ringiti                  321
Test ThunderCloud          3
Name: customerAccountNumber, dtype: int64

In [34]:
hs = df[df['tariff']=='Residential'].groupby(by=["projectName"])["customerAccountNumber"].count()
ci = df[df['tariff'].isin(['Commercial', 'Religious Institution'])].groupby(by=["projectName"])["customerAccountNumber"].count()
hs_count = pd.DataFrame(hs.reset_index())
ci_count = pd.DataFrame(ci.reset_index())
temp = projects.merge(hs_count, how="left", left_on="project_name", right_on="projectName")
temp.rename(mapper={"customerAccountNumber": "Households"}, axis=1, inplace=True)
final = temp.merge(ci_count, how="left", left_on="project_name", right_on="projectName")
final.rename(mapper={"customerAccountNumber": "CI"}, axis=1, inplace=True)
final.drop(columns=["projectName_x", "projectName_y"], axis=1, inplace=True)
final["test"] = final["customer_count"] - (final["Households"] + final["CI"])
final

,project_name,community,country,pv_size_kwh,customer_count,capex_usd,Households,CI,test
0,Kalobeyei Settlement,kalobeyei_settlement,Kenya,541.40,2755,967872,2312,515,-72
1,Ndeda,ndeda,Kenya,23.40,405,197853,332,73,0
2,Emereoke,emereoke,Nigeria,28.08,401,336197,370,31,0
3,Akipelai,akipelai,Nigeria,67.32,355,323701,347,15,-7
4,Locheremoit,locheremoit,Kenya,37.00,329,311220,274,55,0
5,Oloibiri,oloibiri,Nigeria,67.32,321,346464,297,31,-7
6,Ringiti,ringiti,Kenya,20.45,291,142561,256,64,-29
7,Ozuzu,ozuzu,Nigeria,18.72,265,183502,251,17,-3
8,Bendeghe-Afi,bendeghe_afi,Nigeria,18.72,246,218907,278,6,-38
9,Opu,opu,Nigeria,14.04,227,198777,218,9,0
